In [196]:
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import pickle
import sys
import csv

In [197]:
import numpy as np
import pandas as pd
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re as re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score


import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
from tensorflow.keras.layers import Dense, Dropout, Flatten

import tensorflow.keras.backend as K

In [198]:
#ESTO NO LO EJECUTEN SI UTILIZAN UNA VERSION DE PC , YO LO HAGO PARA PODER USAR EL TRAIN DE MI DRIVE,Y USAR EL GITHUB ONLINE
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [199]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1BDGyvJqONz4gk2Y0BK1Ag1DA9yujd44e'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train_uni_loca_3.csv')

In [200]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '18Ab39T1hPgTBv58oXdVk0NfBE36Fp21D'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test_uni_loca_3.csv')


In [201]:
#Cargamos los datos

tweets  = pd.read_csv("train_uni_loca_3.csv")
tweets_test = pd.read_csv("test_uni_loca_3.csv")

In [202]:
#tweets = pd.read_csv('train.csv')
#tweets_test = pd.read_csv('test.csv')

tweets['length']=tweets['text'].str.len()
tweets_test['length']=tweets_test['text'].str.len()

tweets['keyword'] = tweets['keyword'].str.replace('%20', ' ')
tweets['keyword'].fillna('no keyword', inplace = True)

tweets_test['keyword'] = tweets_test['keyword'].str.replace('%20', ' ')
tweets_test['keyword'].fillna('no keyword', inplace = True)

tweets = tweets.sample(frac=1,random_state=1)

In [203]:
tweets['clean_text'] = tweets['text'].str.lower()
tweets_test['clean_text'] = tweets_test['text'].str.lower()

In [204]:
def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

In [205]:
tweets['clean_text'] = tweets['clean_text'].apply(only_letters)
tweets_test['clean_text'] = tweets_test['clean_text'].apply(only_letters)

In [206]:
import nltk
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')

tweets['clean_text'] = tweets['clean_text'].apply(word_tokenize)
tweets_test['clean_text'] = tweets_test['clean_text'].apply(word_tokenize)
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [207]:
def filter_stopwords(tokenized_text):
    not_stopwords=[]
    for w in tokenized_text:
        if w not in stop_words:
            not_stopwords.append(w)
    return not_stopwords

In [208]:
tweets['clean_text'] = tweets['clean_text'].apply(filter_stopwords)
tweets_test['clean_text'] = tweets_test['clean_text'].apply(filter_stopwords)

In [209]:
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [210]:
def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

In [211]:
tweets['clean_text'] = tweets['clean_text'].apply(lemmatize_tweet)
tweets_test['clean_text'] = tweets_test['clean_text'].apply(lemmatize_tweet)
tweets['clean_text'] = tweets['clean_text'].apply(lambda text:' '.join(text))
tweets_test['clean_text'] = tweets_test['clean_text'].apply(lambda text:' '.join(text))
tweets['clean_text'] = tweets['clean_text'].apply(lambda text: re.sub(r'amp | im', '', text))
tweets_test['clean_text'] = tweets_test['clean_text'].apply(lambda text: re.sub(r'amp | im', '', text))

In [212]:
def get_hashtags(s):
    return list(part[1:] for part in s.split() if part.startswith('#'))

In [213]:
tweets['hashtags'] = tweets['text'].apply(get_hashtags)
tweets_test['hashtags'] = tweets_test['text'].apply(get_hashtags)

In [214]:
def hashtag_length_proportion(hashtags,length):
    return len(''.join(hashtags))/length

In [215]:
tweets['len_hash_over_text'] = tweets.apply(lambda data: hashtag_length_proportion(data['hashtags'],data['length']),axis=1)
tweets_test['len_hash_over_text'] = tweets_test.apply(lambda data: hashtag_length_proportion(data['hashtags'],data['length']),axis=1)

In [216]:
def get_mentioned_users(s):
    return list(part[1:] for part in s.split() if part.startswith('@'))

In [217]:
tweets['users'] = tweets['text'].apply(get_mentioned_users)
tweets_test['users'] = tweets_test['text'].apply(get_mentioned_users)

In [218]:
def amount_hashtags(l):
    return len(l)

In [219]:
tweets['amount_hashtags']=tweets['hashtags'].apply(amount_hashtags)
tweets_test['amount_hashtags']=tweets_test['hashtags'].apply(amount_hashtags)

In [220]:
def amount_users(l):
    return len(l)

In [221]:
def has_url(text):
    return int('http' in text)

In [222]:
tweets['has_url']=tweets['text'].apply(has_url)
tweets_test['has_url']=tweets_test['text'].apply(has_url)

In [223]:
def find_url(text):
    urls = re.findall(r'(https?://\S+)', text)
    return urls

In [224]:
tweets['urls']=tweets['text'].apply(find_url)
tweets_test['urls']=tweets_test['text'].apply(find_url)
tweets['amount_urls'] = tweets['urls'].apply(lambda x: len(x))
tweets_test['amount_urls'] = tweets_test['urls'].apply(lambda x: len(x))

In [225]:
tweets['amount_users']=tweets['users'].apply(amount_users)
tweets_test['amount_users']=tweets_test['users'].apply(amount_users)

In [226]:
#Mean encoding keyword
tweets['keyword_encoded'] = tweets.groupby('keyword')['target'].transform('mean')

In [227]:
keywords_dict = pd.Series(tweets['keyword_encoded'].values, index=tweets['keyword']).to_dict()

In [228]:
tweets_test['keyword_encoded']= tweets_test['keyword'].map(keywords_dict)

In [229]:
tweets['len_clean_text'] = tweets['clean_text'].str.len()
tweets_test['len_clean_text'] = tweets_test['clean_text'].str.len()

tweets['len_clean_text_over_text'] = tweets['len_clean_text']/tweets['length']
tweets_test['len_clean_text_over_text'] = tweets_test['len_clean_text']/tweets_test['length']

In [231]:
tweets.head()

,Unnamed: 0,id,keyword,location,text,target,clean_text,length,hashtags,len_hash_over_text,users,amount_hashtags,has_url,urls,amount_urls,amount_users,keyword_encoded,len_clean_text,len_clean_text_over_text
3228,3228,6821,loud bang,Florida,@SW_Trains strange loud impact bang noises und...,0,swtrains strange loudpact bang noise train eps...,90,[Wimbledon],0.1,[SW_Trains],1,0,[],0,1,0.205882,65,0.722222
3706,3706,7840,quarantine,Diego,Loved Chicago so much that it game me Pink Eye...,0,loved chicago much game pink eye sit design qu...,99,[],0.0,[],0,0,[],0,0,0.147059,59,0.595960
6957,6820,7635,pandemonium,NaN,Pandemonium In Aba As Woman Delivers Baby With...,0,pandemonium aba woman delivers baby without fa...,109,[],0.0,[],0,1,"[http://t.co/8j4rdwyjWu, http://t.co/9MkZPZfKL2]",2,0,0.216216,54,0.495413
2887,2887,6074,heat wave,"Maricopa, AZ",@Startide It's hotter there than Phoenix this ...,0,startide hotter phoenix week x humidity gtgt h...,94,[],0.0,[Startide],0,0,[],0,1,0.735294,54,0.574468
7464,7432,10113,upheaval,NaN,A look at state actions a year after Ferguson...,0,look state action year fergusons upheaval,82,[],0.0,[],0,1,[http://t.co/G5ZsRU0zVQ],1,0,0.157895,41,0.500000


In [233]:
tweets_test.head()

,Unnamed: 0,id,keyword,location,text,clean_text,length,hashtags,len_hash_over_text,users,amount_hashtags,has_url,urls,amount_urls,amount_users,keyword_encoded,len_clean_text,len_clean_text_over_text
0,0,46,ablaze,Florida,Birmingham Wholesale Market is ablaze BBC News...,birmingham wholesale market ablaze bbc news fi...,120,[],0.000000,[],0,1,[http://t.co/irWqCEZWEU],1,0,0.361111,82,0.683333
1,1,47,ablaze,Niall's place | SAF 12 SQUAD |,@sunkxssedharry will you wear shorts for race ...,sunkxssedharry wear short race ablaze,54,[],0.000000,[sunkxssedharry],0,0,[],0,1,0.361111,37,0.685185
2,2,51,ablaze,Nigeria,#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriag...,previouslyondoyintv toke makinwas marriage cri...,109,[PreviouslyOnDoyinTv:],0.183486,[],1,1,[http://t.co/CMghxBa2XI],1,0,0.361111,77,0.706422
3,3,58,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,check nsfw,114,[nsfw],0.035088,[],1,1,"[http://t.co/rOI2NSmEJJ, http://t.co/3Tj8ZjiN2...",4,0,0.361111,10,0.087719
4,4,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...,psa splitting personality techie follow ablaze...,94,[],0.000000,"[ablaze_co, ablaze]",0,0,[],0,2,0.361111,69,0.734043


In [234]:
training_set=tweets
testing_set=tweets_test

In [256]:
def make_predictions(model, test_set, final_features, file_name):

#"model: el modelo entrenado
 #  test_set: registros con los features que ayudan a predecir
#         final_features: features usados para entrenar el modelo, si se usaron todos es test_set.columns
#        file_name: nombre del archivo donde guardar los resultados  
# escribe un archivo con (id,target) con las predicciones del modelo y lo guarda con el nombre
#file_name"""
    

    ids=test_set['id'].values
    tweet_value = pd.DataFrame(ids ,columns={'id'})
    predictions = model.predict(test_set[final_features])

    tweet_value['target']=predictions

    from google.colab import files
   
 
    tweet_value.to_csv(file_name,index=False)
    files.download(file_name)
    

In [236]:
class RedNeuronalClassifierParameterSearcher(object):

    def train_and_score(self, training_set, final_features):

        """"Devuelve un array de tuplas, donde el primer elemento de cada tupla es el score para esa combinacion de hiper-parametros,
        se usa cross-validation"""   
        layer_options = [5]
        cantidadDeFeature=len(final_features)
        actual_data = training_set[final_features]
        labels = training_set["target"]
        scores_actuales = []
        for layer in layer_options :
            print ("Ahora probando con: " + str(layer) +" capas de la red")
            #cargo datos al modelo
            
            clf = MLPClassifier(activation='relu',solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(layer,2),batch_size='auto', random_state=1)

            scores = cross_val_score(clf, actual_data, labels, cv=40,  scoring='neg_mean_squared_error', n_jobs = -1) 
            scores_actuales.append((scores.mean(),[layer]))
        return scores_actuales

    def train_and_fit(self, training_set, final_features, hiperparameters):
        
        layer = hiperparameters[0]
        #cargo datos al modelo
        clf = MLPClassifier(activation='relu',solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(layer,2),batch_size='auto', random_state=1)
        clf.fit(training_set[final_features], training_set["target"])    
        return clf


In [237]:
class RandomForestRegressorParameterSearcher(object):

    def train_and_score(self, training_set, final_features):

        """"Devuelve un array de tuplas, donde el primer elemento de cada tupla es el score para esa combinacion de hiper-parametros,
        se usa cross-validation"""   
        sample_leaf_options = [5]
        cantidadDeFeature=len(final_features)
        actual_data = training_set[final_features]
        labels = training_set["target"]
        scores_actuales = []
        for leaf_size in sample_leaf_options :
            print ("Ahora probando con: "+str(leaf_size)+" sample_leaf")
            #cargo datos al modelo
            Forest= RandomForestClassifier(n_estimators = 20, n_jobs = -1,random_state =3 ,max_features = "auto", min_samples_leaf = leaf_size)
            scores = cross_val_score(Forest, actual_data, labels, cv=40,  scoring='neg_mean_squared_error', n_jobs = -1) 
            scores_actuales.append((scores.mean(),[leaf_size]))
        return scores_actuales

    def train_and_fit(self, training_set, final_features, hiperparameters):
        
        leaf_size = hiperparameters[0]
        #cargo datos al modelo
        ForestClassifier= RandomForestClassifier(n_estimators=20, oob_score = True, n_jobs = -1,random_state =50,max_features = "auto", min_samples_leaf = leaf_size)
        ForestClassifier.fit(training_set[final_features], training_set["target"])    
        return ForestClassifier


In [238]:
class DecisionTreeRegressorParameterSearcher(object):

    def train_and_score(self, training_set, final_features):

        """"Devuelve un array de tuplas, donde el primer elemento de cada tupla es el score para esa combinacion de hiper-parametros,
        se usa cross-validation"""    

        actual_data = training_set[final_features]
        labels = training_set["target"]
        scores_actuales = []
        for x in range(5,40):
            print ("Ahora probando con: "+str(x)+" min_samples_split")
            regressor_tree = DecisionTreeClassifier(min_samples_split = x, min_samples_leaf = 15)
            scores = cross_val_score(regressor_tree, actual_data, labels, cv=40, scoring='neg_mean_squared_error', n_jobs = -1)
            scores_actuales.append((scores.mean(),[x]))
        return scores_actuales

    def train_and_fit(self, training_set, final_features, hiperparameters):
        
        min_samples = hiperparameters[0]
        regressor_tree = DecisionTreeClassifier(min_samples_split = min_samples, min_samples_leaf = min_samples)
        regressor_tree.fit(training_set[final_features], training_set["target"])
        return regressor_tree


In [239]:
class KNeighborsRegressorParameterSearcher(object):
    
    def train_and_score(self, training_set, final_features):

        """"Devuelve un array de tuplas, donde el primer elemento de cada tupla es el score para esa combinacion de hiper-parametros,
        se usa cross-validation"""    

        actual_data = training_set[final_features]
        labels = training_set["target"]
        scores_actuales = []
        for x in range(5,30):
            print ("Ahora probando con: "+str(x)+" vecinos")
            knn = KNeighborsClassifier(n_neighbors= x)
            scores = cross_val_score(knn, actual_data, labels, cv=40, scoring='neg_mean_squared_error', n_jobs = -1)
            scores_actuales.append((scores.mean(),[x]))
        return scores_actuales

    def train_and_fit(self, training_set, final_features, hiperparameters):
        n = hiperparameters[0]
        knn = KNeighborsClassifier(n_neighbors = n)
        knn.fit(training_set[final_features], training_set["target"])
        return knn

In [240]:
def find_best_model(training_set, model, final_features, cant_features, original_features):
#training_set: el set de training, completo
#model: el id del modelo a probar (ver arriba los ids implementados)
#final_features: una lista, puede ser vacia o contener inicialmente algun feature que se quiere que se tenga en cuenta
#cant_features: cantidad de features a probar, en un principio siempre es la cantidad de features - 1(id  no se prueban,)

#:return: modelo fitted al training set con los mejores parametros del modelo y mejores features encontrados
    features = list(original_features)
    scores_previos = []
    best_score = -float("inf") #Empiezo con un error infinito
    features_best_score = final_features #
    best_hiper_parameters = []  
    agregar_nuevo_feature = False #Primera iteracion la hago con el primer feature a probar
    modelo_a_entrenar = implemented_models[model]#saca la clase a probar
    feature_actual = 0
    while feature_actual < cant_features:   #este ciclo recorre todas las features columnas(menos id del tweet)
        print ("Inicio de una nueva iteracion")
        print ("Best score actual: "+str(best_score)+", hasta ahora las cols son: "+str(features_best_score)+" y los mejores hiper-parametros son: "+str(best_hiper_parameters))
        if agregar_nuevo_feature:            
            final_features.append(features[feature_actual])

        scores_con_features_actuales = modelo_a_entrenar.train_and_score(training_set, final_features)#al principio tendra el primer feature a probar
        improved = False
        for s in scores_con_features_actuales:#escore por cada hiperparemtro,por eso es un array
            if s[0] > best_score: 
                 best_score = s[0]
                 features_best_score = list(final_features)#ademas guarda la lista de columanas finales
                 best_hiper_parameters = s[1]
                 improved = True
        if not(improved):#si no mejoro elimina esa feature,la primera no se elimnina porqeue siempre es escore es mayor a -inf
            final_features.remove(features[feature_actual])
      
        feature_actual += 1
        scores_previos.append(scores_con_features_actuales)
       
        agregar_nuevo_feature = True #Agrego nuevo feature a partir de la segunda iteracion
        print ("Fin de la ultima iteracion")

    print ("Best score actual: "+str(best_score)+", hasta ahora las cols son: "+str(features_best_score)+" y los mejores hiper-parametros son: "+str(best_hiper_parameters))
    print ("Entrenando el mejor modelo hallado,con el mejor score segun la feature y su hiperparametro")
    best_model = modelo_a_entrenar.train_and_fit(training_set, features_best_score, best_hiper_parameters)
    return best_model,features_best_score#devuelve el modelo entreando y sus mejores columnas


In [241]:
#elegir uno de ellos modelos a entrenar
implemented_models = {"knn":KNeighborsRegressorParameterSearcher(),"decisiontree":  DecisionTreeRegressorParameterSearcher(), "randomforest": RandomForestRegressorParameterSearcher(),"red":RedNeuronalClassifierParameterSearcher()}

In [ ]:
####encontrar los mejores hiperparametros(sin buscar las mejores columnas,osea las mejores features ya vos lo pones)

In [ ]:
training_set.head(5)

,Unnamed: 0,id,keyword,location,text,target,clean_text,length,hashtags,len_hash_over_text,users,amount_hashtags,has_url,urls,amount_urls,amount_users,keyword_encoded,len_clean_text,len_clean_text_over_text
3228,3228,6821,loud bang,Florida,@SW_Trains strange loud impact bang noises und...,0,swtrains strange loudpact bang noise train eps...,90,[Wimbledon],0.1,[SW_Trains],1,0,[],0,1,0.205882,65,0.722222
3706,3706,7840,quarantine,Diego,Loved Chicago so much that it game me Pink Eye...,0,loved chicago much game pink eye sit design qu...,99,[],0.0,[],0,0,[],0,0,0.147059,59,0.595960
6957,6820,7635,pandemonium,NaN,Pandemonium In Aba As Woman Delivers Baby With...,0,pandemonium aba woman delivers baby without fa...,109,[],0.0,[],0,1,"[http://t.co/8j4rdwyjWu, http://t.co/9MkZPZfKL2]",2,0,0.216216,54,0.495413
2887,2887,6074,heat wave,"Maricopa, AZ",@Startide It's hotter there than Phoenix this ...,0,startide hotter phoenix week x humidity gtgt h...,94,[],0.0,[Startide],0,0,[],0,1,0.735294,54,0.574468
7464,7432,10113,upheaval,NaN,A look at state actions a year after Ferguson...,0,look state action year fergusons upheaval,82,[],0.0,[],0,1,[http://t.co/G5ZsRU0zVQ],1,0,0.157895,41,0.500000


In [246]:

features = list(training_set.columns)
#sacamos los features que creemos indispensables

features.remove('Unnamed: 0')
features.remove('text')
features.remove('clean_text')
features.remove('location')
features.remove('keyword')
features.remove('hashtags')
features.remove('users')
features.remove('id')
features.remove('urls')





In [247]:
#cambio para que en la primer posicion este nivel_lenght, ya qye ese no se evaluea
features=['length',
 'len_hash_over_text',
 'amount_hashtags',
 'has_url',
 'amount_urls',
 'amount_users',
 'keyword_encoded',
 'len_clean_text',
 
 'len_clean_text_over_text']

In [248]:
 #Cantidad de features a probar con forward selection:
cant_features = len(features)  
#, target es lo que quiero predecir (target), ni rooms ni superficie total (estos dos features no se descartan nunca)
final_features = ["length"] #fija que esta, tiene alta correlacion 

In [249]:
##elegis el modelo , ejemplo knn decisiontree
model="randomforest"
#nombre del archivo done se guardara la prediccion
file_name=model

In [251]:
if model.lower() not in implemented_models:
    print ("No se ha implementado aun el modelo: "+model)
    

In [252]:
info_entrenamiento = find_best_model(training_set, model, final_features, cant_features, features)
#te devuelve un array de dos dimentsiones, en la primera el modelo, y en la segunda las mejores columnas
best_model = info_entrenamiento[0]
best_features = info_entrenamiento[1]



Inicio de una nueva iteracion
Best score actual: -inf, hasta ahora las cols son: ['length'] y los mejores hiper-parametros son: []
Ahora probando con: 5 sample_leaf
Fin de la ultima iteracion
Inicio de una nueva iteracion
Best score actual: -0.3999848443097272, hasta ahora las cols son: ['length'] y los mejores hiper-parametros son: [5]
Ahora probando con: 5 sample_leaf
Fin de la ultima iteracion
Inicio de una nueva iteracion
Best score actual: -0.39932626067787264, hasta ahora las cols son: ['length', 'len_hash_over_text'] y los mejores hiper-parametros son: [5]
Ahora probando con: 5 sample_leaf
Fin de la ultima iteracion
Inicio de una nueva iteracion
Best score actual: -0.39932626067787264, hasta ahora las cols son: ['length', 'len_hash_over_text'] y los mejores hiper-parametros son: [5]
Ahora probando con: 5 sample_leaf
Fin de la ultima iteracion
Inicio de una nueva iteracion
Best score actual: -0.3580855607605401, hasta ahora las cols son: ['length', 'len_hash_over_text', 'has_url'

In [253]:
best_features

['length',
 'len_hash_over_text',
 'has_url',
 'amount_users',
 'keyword_encoded',
 'len_clean_text']

In [257]:
make_predictions(best_model, testing_set, best_features, file_name+".csv")
pickle.dump(best_model, open(file_name+".sav", 'wb'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>